In [ ]:
#Import Dependencies
from splinter import Browser
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import pymongo

## NASA Mars News

In [ ]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/'

In [ ]:
# Retrieve page with the requests module
response = requests.get(url)
#response.headers
#response.content

In [ ]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
# Examine the results, then determine element that contains sought info
#print(soup.prettify())

In [ ]:
# First paragraph result returned for first article on page
news_paragraph = soup.find('div', class_="rollover_description_inner").text.strip()
news_paragraph

In [ ]:
# First title result returned for first article on page
news_title = soup.find('div', class_="content_title").a.text.strip()
news_title

## JPL Mars Space Images - Featured Image


In [ ]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [ ]:
# Scrape page into Soup
html = browser.html
image_soup = BeautifulSoup(html, 'html.parser')
image_elem = image_soup.find_all('div', class_='carousel_items')

In [ ]:
# Iterate through the feature image botton
feature_image_botton = image_elem[0].find('a', class_="button fancybox")
feature_image_botton

In [ ]:

# Find the website source of the image

browser.links.find_by_partial_text('FULL IMAGE').click()
browser.links.find_by_partial_text('more info').click()


In [ ]:
#Delay the extraction of information since the website took several seconds to load completely
browser.is_element_present_by_css("figure.lede", wait_time=1)

In [ ]:

# Store the html of the current website
image_html = browser.html

In [ ]:

# Scrape page into Soup
image_soup = BeautifulSoup(image_html, 'html.parser')

In [ ]:

# Retrieve the figure tag 
figure_tag= image_soup.find_all('figure', class_='lede')

In [ ]:
# Retrieve the image source from figure tag
image_source = figure_tag[0].find('a')['href']


In [ ]:
image_source

In [ ]:

# Store the image url 

website_url = 'https://www.jpl.nasa.gov'
featured_image_url = website_url + image_source
featured_image_url

## Mars Facts

In [ ]:
# Use spliter to obtain the information from the website 
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path)
url = 'https://space-facts.com/mars/'
browser.visit(url)


In [ ]:
# Use pandas to read the html table data on the page into a list of dictionaries
tables = pd.read_html(url)
tables

In [ ]:
# Read the first dictionary in the list into a pandas dataframe and name columns
df = tables[0]
df.columns = ['Parameter', 'Value']

In [ ]:
df.set_index('Parameter', inplace=True)
df

In [ ]:

# Convert the dataframe into an html table, strip the end of line newlines and 
# write the result to an html file to view 
fact_table = df.to_html()
fact_table = fact_table.replace('\n', '')
fact_table

In [ ]:
# Inspect the result in a browser
df.to_html('table.html')
!explorer table.html

## Mars Hemispheres

In [ ]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

In [ ]:
# Get page html and make beautifulsoup object
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
# Get the html containing the titles and put into a list
title_list = soup.find_all('div', class_='description')
title_list

In [ ]:
# Loop through the div objects and scrape titles and urls of hires images
# Initiate the list to store dictionaries
hemisphere_image_urls = []
for title in title_list:
    # Navigate browser to page then click on title link to hires image page
    browser.visit(url)
    browser.click_link_by_partial_text(title.a.h3.text)

    # Grab the destination page html and make into BeautifulSoup object
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    # Parse the hires image source(src) relative url then append to domain name
    # for absolute url 
    img_url_list = soup.find('img', class_='wide-image')
    img_url = f"https://astrogeology.usgs.gov{img_url_list['src']}"

    # Create dictionary with returned values and add dict to hemisphere_image_urls list
    post = {
            'title': title.a.h3.text,
            'image_url': img_url
            }
    hemisphere_image_urls.append(post)

In [ ]:
hemisphere_image_urls